In [1]:
import pandas as pd
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib as plt
import pickle
import statistics

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df_jan = pq.read_table('yellow_tripdata_2023-01.parquet').to_pandas()

In [4]:
df_feb = pq.read_table('yellow_tripdata_2023-02.parquet').to_pandas()

In [5]:
df_jan.shape

(3066766, 19)

In [6]:
df_feb.shape

(2913955, 19)

In [7]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [8]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [9]:
df_jan['duration'] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).dt.total_seconds()/60

In [10]:
df_jan['PULocationID'] =df_jan['PULocationID'].astype(str)

In [11]:
df_jan['DOLocationID'] =df_jan['DOLocationID'].astype(str)

In [12]:
statistics.stdev(df_jan.duration)

42.59435124195457

In [13]:
(df_jan['duration']>=1) & (df_jan['duration']<=60)

0          True
1          True
2          True
3          True
4          True
           ... 
3066761    True
3066762    True
3066763    True
3066764    True
3066765    True
Name: duration, Length: 3066766, dtype: bool

In [14]:
df_jan_model = df_jan[(df_jan['duration']>=1) & (df_jan['duration']<=60)]

In [15]:
df_jan_model.shape[0]/df_jan.shape[0]

0.9812202822125979

In [16]:
df_jan_model[:10].to_dict(orient = 'records')

[{'VendorID': 2,
  'tpep_pickup_datetime': Timestamp('2023-01-01 00:32:10'),
  'tpep_dropoff_datetime': Timestamp('2023-01-01 00:40:36'),
  'passenger_count': 1.0,
  'trip_distance': 0.97,
  'RatecodeID': 1.0,
  'store_and_fwd_flag': 'N',
  'PULocationID': '161',
  'DOLocationID': '141',
  'payment_type': 2,
  'fare_amount': 9.3,
  'extra': 1.0,
  'mta_tax': 0.5,
  'tip_amount': 0.0,
  'tolls_amount': 0.0,
  'improvement_surcharge': 1.0,
  'total_amount': 14.3,
  'congestion_surcharge': 2.5,
  'airport_fee': 0.0,
  'duration': 8.433333333333334},
 {'VendorID': 2,
  'tpep_pickup_datetime': Timestamp('2023-01-01 00:55:08'),
  'tpep_dropoff_datetime': Timestamp('2023-01-01 01:01:27'),
  'passenger_count': 1.0,
  'trip_distance': 1.1,
  'RatecodeID': 1.0,
  'store_and_fwd_flag': 'N',
  'PULocationID': '43',
  'DOLocationID': '237',
  'payment_type': 1,
  'fare_amount': 7.9,
  'extra': 1.0,
  'mta_tax': 0.5,
  'tip_amount': 4.0,
  'tolls_amount': 0.0,
  'improvement_surcharge': 1.0,
  'tota

In [17]:
df_jan_model.shape

(3009173, 20)

In [18]:
df_jan_model.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [ ]:
df_dict = df_jan_model.to_dict(orient = 'records')

In [ ]:
df_jan = pq.read_table('yellow_tripdata_2023-01.parquet')

df = df.to_pandas()

df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

df['duration'] = df.lpep_dropoff_datetime-df.lpep_pickup_datetime

df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

categorical = ['PULocationID', 'DOLocationID']

numerical = ['trip_distance']

df[categorical].dtypes

df[categorical] = df[categorical].astype(str)

In [ ]:
df = df.to_pandas()

In [ ]:
df.head()

In [ ]:
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

In [ ]:
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [ ]:
df['duration'] = df.lpep_dropoff_datetime-df.lpep_pickup_datetime

In [ ]:
df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

In [ ]:
sns.distplot(df.duration)

In [ ]:
df.duration.describe(percentiles=[0.95, ])

In [ ]:
df = df[(df.duration >=1)& (df.duration <=60)]

In [ ]:
df.head()

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

In [ ]:
numerical = ['trip_distance']

In [ ]:
df[categorical].dtypes

In [ ]:
df[categorical] = df[categorical].astype(str)

In [ ]:
train_dict = df[categorical + numerical].to_dict(orient = 'records')

In [ ]:
dv = DictVectorizer()

In [ ]:
X_train = dv.fit_transform(train_dict)

In [ ]:
X_train

In [ ]:
dv

In [ ]:
dv.feature_names_

In [ ]:
target = 'duration'
y_train = df[target].values

In [ ]:
y_train

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_train)

In [ ]:
sns.distplot(y_pred, label ='prediction')
sns.distplot(y_train, label ='actual')
plt.pyplot.legend()

In [ ]:
mean_squared_error(y_train, y_pred, squared = False)

In [ ]:
with open('model/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
# Code restructuring

def read_dataframe(filename):
    df = pq.read_table('green_tripdata_2021-01.parquet')

    df = df.to_pandas()

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime-df.lpep_pickup_datetime

    df['duration'] = df.duration.apply(lambda td: td.total_seconds()/60)

    categorical = ['PULocationID', 'DOLocationID']

    numerical = ['trip_distance']

    df[categorical].dtypes

    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('green_tripdata_2021-01.parquet')
df_test = read_dataframe('green_tripdata_2021-02.parquet')

In [ ]:
dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.fit_transform(val_dict)

In [ ]:
target